In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# server의 데이터를 불러옴
drug = pd.read_csv('/mnt/towel/UKBiobank/Clinical/41770/GP/gp_scripts.txt', sep='\t', 
                            encoding='ISO-8859-1', engine='python')

In [ ]:
main = pd.read_csv('/mnt/towel/UKBiobank/Clinical/39581/MAIN/ukb39581.csv')

In [ ]:
# 진단 데이터를 불러옴
diagnosis = pd.read_csv('/mnt/towel/UKBiobank/Clinical/41770/HES/hesin_diag.txt', sep='\t', 
                        encoding='ISO-8859-1', engine='python')

In [ ]:
# 진단 데이터의 환자수를 불러옴[8659522 rows]-> [413572 rows]
diagnosis.drop_duplicates(['eid']) # 중복 데이터를 제거 함
diagnosis = diagnosis.dropna(subset = ['diag_icd10']) # icd10의 결측치를 제거 [8587266 rows] ->410319 [410319 patiens]
diagnosis.drop_duplicates(['eid'])

# 당뇨병 환자만을 추출 시행(diagnosis)
* diagnosis 데이터들중에서 당뇨병 환자만을 추출 시행 (type2 diabetes)
* 해당하는 icd 10 code는 E11을 가짐 (E11중 icd9 code를 가지는 데이터는 없음)
* icd 9를 기준으로도 분석을 시행 (따로 시행)

In [ ]:
# icd 10 기준
diabetes_filter = diagnosis['diag_icd10'].str.contains('E11')
diabetes_diag = diagnosis[diabetes_filter] #159155 rows -> 29267 rows patients
################################################
diabetes_plist = list(diabetes_diag.drop_duplicates(['eid']).eid) #당뇨병 환자의 아이디 리스트 생성!!!!!1

In [ ]:
# icd 9 기준 (없음)
diagnosis = pd.read_csv('/mnt/towel/UKBiobank/Clinical/41770/HES/hesin_diag.txt', sep='\t', 
... encoding='ISO-8859-1', engine='python')
diabetes_filter_2 = diagnosis['diag_icd9'].str.contains('E11')
diabetes_diag_2 = diagnosis[diabetes_filter_2]
diabetes_diag_2 # Index: []-> 없음

# 당뇨병 환자들 중 췌장염, 췌장암 환자들 추출
* 췌장염과 췌장암의 진단을 받은 환자들 추출을 시행
* 당뇨병 환자와의 교집합의 아이디 리스트를 추출
* 진단 선후관계 파악 

In [ ]:
# icd 10 기준 총 537명이 존재함
diabetes_diag3 = diagnosis[diagnosis['eid'].isin(diabetes_plist)]  #당뇨병을 가진 환자의 다른 진단도 포함한 데이터 형성
pan_filter = diabetes_diag3['diag_icd10'].str.contains('K85|K86') 
pan_diag = diabetes_diag3[pan_filter]
# K860, K852는 알콜 유도성임으로 제외를 시킴!!!!!!!!!!!!!!!!!!!
filter = pan_diag['diag_icd10'] != 'K860'
pan_diag2 = pan_diag[filter]
filter = pan_diag['diag_icd10'] != 'K852'
pan_diag3 = pan_diag2[filter]
pan_diag = pan_diag3   #3492 -> 804 patiens  ------------>다시 시작해야함!!!!!!!!!!!!!!!!
############################################################당뇨병과 췌암질환을 모두 가지는 환자 리스트!!!!!!
############################
pan_plist = list(pan_diag.drop_duplicates(['eid']).eid)

In [ ]:
# 췌장, 당뇨의 선후 관계 파악
# ins_index의 순서를 사용하여 선후 관계를 파악함 먼저 발생하면 ins_index의 수가 작음!!!!!!!!!!!!!!!
panda = diagnosis[diagnosis['eid'].isin(pan_plist)] 
#당뇨병과 췌장을 모두 가지는 모든 처방 내역(pan_diag는 췌장질환만 존재하여 비교 불가)

In [ ]:
#췌장질환 있는 환자 리스트(pan_plist)중 환자 아이디 한개 추출해서 분석
# ins_index의 수가 증가 할 수록 늦게 시작됨을 의미
panda_diag = panda[panda['diag_icd10'].str.contains('K85|K86|E11')]
panda_e = panda_diag[panda_diag['diag_icd10'].str.contains('E')]  
panda_e = panda_e.drop_duplicates(['eid']) # 당뇨병 환자들 중복값 제거 어차피 선후 관계만 파악하는거니
panda_e.set_index('eid', inplace=True) #인덱스 아이디로 시행
panda_e = panda_e[['ins_index', 'diag_icd10']]
panda_e.columns = ['ins_index2', 'diag_icd102'] # 병합을 위해 column명을 바꿔줌

panda_k = panda_diag[panda_diag['diag_icd10'].str.contains('K')]  
panda_k = panda_k.drop_duplicates(['eid']) # 췌장 환자들 중복값 제거
panda_k.set_index('eid', inplace=True)
panda_k = panda_k[['ins_index', 'diag_icd10']]

#################당뇨병이 먼저 걸리면 ins_index(k) - ins_index2(e) >0 의 양수값을 가짐~!!!
panda_w =panda_k.join(panda_e, how = 'inner') # 공통된 부분을 기점으로 합침 (537환자!!!)
panda_w['cal'] = panda_w['ins_index'] - panda_w['ins_index2']
filter = panda_w['cal']>0
fin = panda_w[filter]
fin
###############################################################당뇨병-> 췌장질환############
fin_list = list(fin.index) # 409명의 환자 아이디를 추출 시행함

# 약물 처방 데이터 
* 당뇨병 환자의 아이디 리스트를 약물 데이터에 적용을 시행함
* 57706541처방 중 10249366처방건수 존재
* 222113환자들 중 12787환자가 당뇨병 환자임

In [ ]:
# 약물 데이터 57706541-> 222113
drug.drop_duplicates(['eid']) #환자별로 중복값을 제거한 후 시행
diabetes_drug = drug[drug['eid'].isin(diabetes_plist)] # 10249366처방이 존재함
diabetes_drug.drop_duplicates(['eid'])  # 12787의 당뇨병 환자들이 처방 데이터가 존재함을 알 수 있음

# 당뇨병 환자 처방데이터들 중 anti-diabetes 약물 처방 수 분석
## dpp-4사용 처방, dpp-4처방하지 않은 환자군을 설정


In [ ]:
# 약물이름 중 결측지 제거를 시행함(total) 
diabetes_drug = diabetes_drug.dropna(subset = ['drug_name']) # 8919371처방 존재
diabetes_drug.drop_duplicates(['eid']) #11573 환자 존재

# dpp_lisst, notdpp_list 에서 glp처방 받은 환자를 제거 시행! (exclusion criteria)
glp_filter = diabetes_drug['drug_name'].str.contains('tide')
glp_drug = diabetes_drug[glp_filter]
glp_drug
glp_list = list(glp_drug.drop_duplicates(['eid']).eid) # 1498환자 존재

not_glp = ~diabetes_drug['eid'].isin(glp_list)
diabetes_drug = diabetes_drug[not_glp]
diabetes_drug
diabetes_drug.drop_duplicates(['eid']) # 10075 patients 존재 (11573 - 1498)

In [ ]:
# 당뇨병 환자들 중 dpp dpp를 처방 받은 환자들
ddp_filter = diabetes_drug['drug_name'].str.contains('gliptin')
ddp_drug = diabetes_drug[ddp_filter]
ddp_drug # 40765처방이 존재
ddp_drug.drop_duplicates(['eid']) # 1642환자 
####################################
ddp_list = list(ddp_drug.drop_duplicates(['eid']).eid)  #당뇨병 환자중 dpp-4를 처방받은 환자 아이디 리스트 & metformin 복합포함)
ddp_drug = diabetes_drug['eid'].isin(ddp_list)
ddp_drug = diabetes_drug[ddp_drug] # 해당 리스트 환자들 처방 데이터를 추출 시행 1985857처방 존재

In [ ]:
# 당뇨병 환자들중 dpp를 처방 받지 않은 처방들
notdpp_drug = ~diabetes_drug['eid'].isin(ddp_list)
notdpp_drug = diabetes_drug[notdpp_drug]
notdpp_drug #5641718처방 존재 차이는 동일 환자 여도 글립틴 외의 약물이 있어서 10249366과 차이 존재함
notdpp_drug.drop_duplicates(['eid']) #8433 환자존재
#############################
notddp_list = list(notdpp_drug.drop_duplicates(['eid']).eid) # 당뇨병 환자들중 dpp-4를 처방 받지 않은 환자

## 다른 당뇨병 약물들 처방 비율을 확인 (dpp 처방, 처방 x -> 췌장염)
* 인슐린 처방
* GI use
* Biguanide use
* Meglitinide
* Sulfonylurea
* TZD

In [ ]:
# TZD drug(total 47701, 1279 patients )
thiazoli_filter = diabetes_drug['drug_name'].str.contains('glitazone')
thiazoli_drug = diabetes_drug[thiazoli_filter]
thiazoli_drug
# 환자 중복 아이디 제거
thiazoli_drug.drop_duplicates(['eid'])

# TZD drug(dpp 처방하지 않은 환자들 31426,794 )
thiazoli_filter = notdpp_drug['drug_name'].str.contains('glitazone')
thiazoli_drug = notdpp_drug[thiazoli_filter]
thiazoli_drug
thiazoli_drug.drop_duplicates(['eid'])

# TZD drug(dpp 처방 환자들 16275, 485)
thiazoli_filter = ddp_drug['drug_name'].str.contains('glitazone')
thiazoli_drug = ddp_drug[thiazoli_filter]
thiazoli_drug
thiazoli_drug.drop_duplicates(['eid'])


In [ ]:
# sglt drug(total 3849,332 )
sglt_filter = diabetes_drug['drug_name'].str.contains('flozin')
sglt_drug = diabetes_drug[sglt_filter]
sglt_drug
sglt_drug.drop_duplicates(['eid'])

# TZD drug(dpp 처방하지 않은 환자들 1692, 139)
sglt_filter = notdpp_drug['drug_name'].str.contains('flozin')
sglt_drug = notdpp_drug[sglt_filter]
sglt_drug
sglt_drug.drop_duplicates(['eid'])

# TZD drug(dpp 처방 환자들 2157, 193)
sglt_filter = ddp_drug['drug_name'].str.contains('flozin')
sglt_drug = ddp_drug[sglt_filter]
sglt_drug
sglt_drug.drop_duplicates(['eid'])


In [ ]:
# Insulin(total 3921,173)
insulin_filter = diabetes_drug['drug_name'].str.contains('Insulin')
insulin_drug = diabetes_drug[insulin_filter]
insulin_drug
insulin_drug.drop_duplicates(['eid'])

# Insulin(dpp 처방하지 않은 환자들 3726,154)
insulin_filter = notdpp_drug['drug_name'].str.contains('Insulin')
insulin_drug = notdpp_drug[insulin_filter]
insulin_drug
insulin_drug.drop_duplicates(['eid'])

# Insulin(dpp 195, 19)
insulin_filter = ddp_drug['drug_name'].str.contains('Insulin')
insulin_drug = ddp_drug[insulin_filter]
insulin_drug
insulin_drug.drop_duplicates(['eid'])


In [ ]:
# MG(2745, 105)
mg_filter = diabetes_drug['drug_name'].str.contains('glinide')
mg_drug = diabetes_drug[mg_filter]
mg_drug
mg_drug.drop_duplicates(['eid'])

# mg drug( not dpp 1392, 57)
mg_filter = notdpp_drug['drug_name'].str.contains('glinide')
mg_drug = notdpp_drug[mg_filter]
mg_drug
mg_drug.drop_duplicates(['eid'])

# mg drug(dpp 1353, 48 )
mg_filter = ddp_drug['drug_name'].str.contains('glinide')
mg_drug = ddp_drug[mg_filter]
mg_drug
mg_drug.drop_duplicates(['eid'])

# dpp-4 처방 환자들 중 췌장질환 발생 빈도 계산
* 약물 처방 리스트 (ddp_list, notddp_list)와 당뇨병, 췌장질환 환자 리스트 교집합 구함

In [ ]:
# dpp처방 받은 환자군들 중 췌장염이 발생한 비율
dpp_pan_list = list(set(fin_list) & set(ddp_list))
len(dpp_pan_list) # 16명
len(dpp_pan_list)/ len(ddp_list)  0.0097442143727162
    
dpp_notpan_list = list(set(ddp_list) - set(fin_list))

# dpp처방 받지 않은 환자군들 중 췌장염이 발생한 비율
notdpp_pan_list = list(set(fin_list) & set(notddp_list))
len(notdpp_pan_list) #102명
len(notdpp_pan_list)/ len(notddp_list) 0.0120

In [ ]:
# 약물 처방 리스트를 생성
thiazoli_list = list(thiazoli_drug.drop_duplicates(['eid']).eid)
sglt_list = list(sglt_drug.drop_duplicates(['eid']).eid)
insulin_list = list(insulin_drug.drop_duplicates(['eid']).eid)
mg_list = list(mg_drug.drop_duplicates(['eid']).eid)

# 단순히 분석 빈도 분석이셔어 여러 confounding factor을 고려하여 위험도 분석을 시행해봐야 할듯 싶음!

# 관련 환자들을 대상으로 confounding factor분석 시행
* BMI
* Alchol
* bileduct disease
* 

In [ ]:
# history of gall bladder and common bile duct stone(CBD) [ICD-10 K80]
stone_filter = diagnosis['diag_icd10'].str.contains('K80')
stone_diag = diagnosis[stone_filter]
stone_plist = list(stone_diag.drop_duplicates(['eid']).eid)
len(stone_plist)

len(list(set(notddp_list) & set(stone_plist))) #655 not dpp & stone
len(list(set(ddp_list) & set(stone_plist))) #141 dpp & stone
len(list(set(dpp_pan_list) & set(stone_plist))) #6  dpp pan 중 stone
len(list(set(dpp_notpan_list) & set(stone_plist))) # 135

# hypertriglyceridemia history (E781)
hyper_filter = diagnosis['diag_icd10'].str.contains('E781')
hyper_diag = diagnosis[stone_filter]
hyper_plist = list(stone_diag.drop_duplicates(['eid']).eid)
len(hyper_plist)
len(list(set(notddp_list) & set(hyper_plist)))
len(list(set(ddp_list) & set(hyper_plist)))
len(list(set(dpp_pan_list) & set(hyper_plist))) #6  dpp pan 중 hyper
len(list(set(dpp_notpan_list) & set(hyper_plist))) #135  dpp not pan & hyper

# anti-diabetes drug ratio of dpp 
len(list(set(dpp_pan_list) & set(thiazoli_list)))
len(list(set(dpp_notpan_list) & set(thiazoli_list)))

len(list(set(dpp_pan_list) & set(sglt_list)))
len(list(set(dpp_notpan_list) & set(sglt_list)))

len(list(set(dpp_pan_list) & set(insulin_list)))
len(list(set(dpp_notpan_list) & set(insulin_list)))

len(list(set(dpp_pan_list) & set(mg_list)))
len(list(set(dpp_notpan_list) & set(mg_list)))

In [ ]:
hesin = pd.read_csv('/mnt/towel/UKBiobank/Clinical/41770/HES/hesin.txt', sep='\t', encoding='ISO-8859-1', engine='python')

# 비율 검정
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

count = np.array([100, 170])
nobs = np.array([300, 400])

z, p = proportions_ztest(count=count, nobs=nobs, value=0)
print(z)
print(p)


# 관련되어진 환자들 데이터를 raw data를 뜯어보기
* dpp_pan_list의 16명의 환자들의 처방 데이터를 모두 살펴 보기
* fin_list 409명도 살펴보기 시행

In [ ]:
# dpp_pan list의 환자들 데이터를 살펴 보기
for i in range(len(dpp_pan_list)+1):
...      print(diagnosis[diagnosis['eid'] == dpp_pan_list[i]])

